In [1]:
import observational_fear.load as load
from observational_fear.nbox_transforms import summarize_prepost_events, wilcoxon_by_cell
from observational_fear.events import get_block_starts
from observational_fear.stats import auc
from neurobox.wide_transforms import resample
from pathlib import Path


DATA_DIR = Path(r"D:\OFL\one-p experiment\data")
FIG_DIR = Path(r"C:\Users\roryl\repos\observational-fear\figs")

In [2]:
SESSION = "day4"
df = load.load_traces(DATA_DIR, session=SESSION)
df = resample(df.set_index("time"), "100ms")
df = df.reset_index().melt(id_vars=["time"])

c:\users\roryl\repos\neurobox\neurobox\wide_transforms.py:99: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df["time"] = pd.to_timedelta(df.index, unit="s")


In [3]:
events = get_block_starts(SESSION, "US")
df_res = summarize_prepost_events(
    df=df, 
    events=events,
    t_before=5, 
    t_after=2,
    summary_func=auc,
    cell_col="new_id"
    )

df_res = wilcoxon_by_cell(df_res, cell_col="new_id")
pct = (df_res.p < 0.05).mean() * 100
num = (df_res.p < 0.05).sum()
print(f"Num Responders: {num} ({pct}%)")

Num Responders: 866 (70.57864710676446%)


In [4]:
df_res.reset_index().assign(block="US").to_csv(DATA_DIR / "day4-block-responders.csv", index=False)